In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)

/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sunxin/minico

1.13.2
2.2.4


Using TensorFlow backend.


In [2]:
import sys
import numpy as np
from keras import layers
from keras import models
from keras import optimizers
from keras.callbacks import LambdaCallback

In [3]:
# https://s3.amazonaws.com/text-datasets/nietzsche.txt
with open("data/nietzsche.txt", encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

corpus length: 600893


In [4]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 57


In [5]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

nb sequences: 200285
Vectorization...


In [6]:
# build the model: a single LSTM
print('Build model...')
model = models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

optimizer = optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...
Instructions for updating:
Colocations handled automatically by placer.


In [7]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [8]:
def on_epoch_end(epoch, _):
    if epoch % 5 != 0:
        return
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = np.random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [9]:
model.fit(x, y,
          batch_size=128,
          epochs=21,
          callbacks=[print_callback])

Instructions for updating:
Use tf.cast instead.
Epoch 1/21
200285/200285 [==============================] - 76s 380us/step - loss: 2.0008

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "y: "thus shall it be!"
they determine fi"
y: "thus shall it be!"
they determine final to the stand and still to the stand the contration of the still the say the soul as the stand and most the stands and the seem and the stand and the comparity and the so the some contrating to the some compurate the most the soul the subject the still as the stand and the still and possity and possity and the soul the contrable and the soul of the soul the stand and the stand and morality and 
----- diversity: 0.5
----- Generating with seed: "y: "thus shall it be!"
they determine fi"
y: "thus shall it be!"
they determine final one such its artive contreally and siffers and again as the so have the morality in an experience and unon the belief ever the belisition to as power, the 

/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


stronges of the conceals and the extent of the strange of the conceales and so the endiration of the strength of the sense of the strange of the consequently the false of the problem of the sense of the soul of the spirit and consequently precisely and the superfiction of the artictive the endire to the word to the soul of the more and sentiment and the more and and th
----- diversity: 0.5
----- Generating with seed: "all men to follow. this evinces
much sim"
all men to follow. this evinces
much simple of the most standing in the strength and are of the artictates and recognize of the modern instrament of the artion in the mode of the specially and of the seise of the more refrist that the corprease of him. he is a superficiate of his religious too harping and consciously that is a seem the orudation of the conceal? when was a self-self-compences of the most suffering and consequently and as
----- diversity: 1.0
----- Generating with seed: "all men to follow. this evinces
much sim"
all

proterress the subjection of the consequence of the provented to the problem of the consequence of the desires of the sense of his which, and the sense of the act of the prose of the prose of the subject, and the consequence of the consequence is the provention of the provented to the provention of the provented to the daise of the sense of the provented to the prose of the more the sense of the proven
----- diversity: 0.5
----- Generating with seed: "sharp, and tender--yes, it is well
prote"
sharp, and tender--yes, it is well
proterred to the truths: in so far to despect is the highest are to which, and be indicate into the propagations, in the progress of the highest thinking we fool for his fellow as a case the possesse of superiority of the world the highest as a contemptation, the means of individually, a cause of the consciousness of secret which has always at all belongs the great of our otter of the former of the res
----- diversity: 1.0
----- Generating with seed: "sharp, and 